In [1]:
# Cell [0]
# ======================================
# 1. Install/Import Required Libraries
# ======================================

import pandas as pd
from google_play_scraper import Sort, reviews
import subprocess
from tqdm import tqdm

In [2]:
# Cell [1]
# ======================================
# 2. Define Parameters
# ======================================

# Package name of the BMW app on Google Play:
app_id = "de.bmw.connected.mobile20.row"

# Ollama model
ollama_model_name = "qwen2.5:latest"

# Prompt template for sentiment classification.
ollama_prompt_template = """You are a sentiment classifier. Classify the sentiment of the following text as Positive, Negative, or Neutral. Do not add any additional information.
Text: "{review_text}"
Answer:
"""

In [3]:
# Cell [2]
# ======================================
# 3. Fetch Reviews from Google Play Store
# ======================================

# Define languages to fetch (just language codes and labels)
languages = [
    ('en', 'English'),
    ('de', 'German'),
    ('fr', 'French'),
    ('it', 'Italian'),
    ('es', 'Spanish'),
    ('nl', 'Dutch'),
    ('sv', 'Swedish'),
    ('da', 'Danish'),
    ('no', 'Norwegian'),
    ('fi', 'Finnish'),
    ('pl', 'Polish'),
    ('cs', 'Czech'),
    ('pt', 'Portuguese'),
    ('zh', 'Chinese'),
    ('ja', 'Japanese'),
    ('ko', 'Korean'),
    ('ar', 'Arabic'),
    ('tr', 'Turkish'),
    ('ru', 'Russian'),
    ('he', 'Hebrew'),
    ('th', 'Thai'),
    ('vi', 'Vietnamese'),
    ('hi', 'Hindi'),
    ('el', 'Greek'),
    ('hu', 'Hungarian'),
    ('ro', 'Romanian'),
    ('sk', 'Slovak'),
    ('bg', 'Bulgarian'),
    ('hr', 'Croatian'),
    ('sr', 'Serbian'),
    ('uk', 'Ukrainian'),
    ('id', 'Indonesian'),
    ('ms', 'Malay'),
    ('fa', 'Persian'),
    ('ur', 'Urdu'),
    ('bn', 'Bengali'),
    ('ta', 'Tamil'),
    ('te', 'Telugu'),
    ('ml', 'Malayalam'),
    ('et', 'Estonian'),
    ('lv', 'Latvian'),
    ('lt', 'Lithuanian'),
    ('sl', 'Slovenian')
]

# Initialize empty list to store all reviews
all_reviews = []

# Fetch reviews for each language
for lang_code, lang_label in languages:
    continuation_token = None
    prev_length = len(all_reviews)
    
    while True:
        result, continuation_token = reviews(
            app_id,
            lang=lang_code,
            sort=Sort.NEWEST,
            count=100,
            continuation_token=continuation_token
        )
        
        # Add language label to each review
        for review in result:
            review['language'] = lang_label
        
        all_reviews.extend(result)
        
        # Break if no more reviews or if number of reviews isn't increasing
        current_length = len(all_reviews)
        if not continuation_token or current_length - prev_length < 100:
            break
            
        prev_length = current_length

# Convert all reviews into a pandas DataFrame
df = pd.DataFrame(all_reviews)

print("\nReview Statistics:")
print("=" * 50)
print(f"Total number of reviews collected: {len(df)}")
print("\nBreakdown by language:")
print("-" * 50)
language_counts = df['language'].value_counts()
print(language_counts)
print("-" * 50)
print(f"Number of languages with reviews: {len(language_counts)}")
# Inspect the first few rows
df.head()


Review Statistics:
Total number of reviews collected: 18328

Breakdown by language:
--------------------------------------------------
language
German        4912
English       4611
French        1709
Italian       1315
Dutch          971
Spanish        966
Polish         638
Portuguese     575
Russian        472
Romanian       286
Swedish        260
Norwegian      204
Japanese       201
Finnish        175
Czech          158
Greek          118
Hungarian      114
Danish         102
Thai            98
Turkish         65
Croatian        65
Slovak          58
Slovenian       53
Chinese         47
Bulgarian       43
Arabic          32
Serbian         20
Ukrainian       14
Lithuanian      12
Estonian         9
Indonesian       7
Latvian          7
Korean           4
Hebrew           4
Malay            2
Persian          1
Name: count, dtype: int64
--------------------------------------------------
Number of languages with reviews: 36


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,language
0,feee29ae-57c2-45fc-99a9-f77248e283de,Erkut Dervish,https://play-lh.googleusercontent.com/a-/ALV-U...,"It is a very nice application, I recommend it ...",5,0,5.3.3,2025-04-08 15:57:55,None,NaT,5.3.3,English
1,1c22dd31-a07c-4aab-a421-793047f28fba,T Cagri Erarslan,https://play-lh.googleusercontent.com/a-/ALV-U...,"Had problems adding my car but after that, it'...",5,0,5.3.3,2025-04-08 13:00:24,Hi there! We are glad to hear that you like th...,2025-04-08 12:48:30,5.3.3,English
2,7ee4cfd3-2af0-4d2f-93bd-3e5b22fdac29,A Halma,https://play-lh.googleusercontent.com/a/ACg8oc...,slow and a complete battery hog. we would be b...,1,0,5.3.3,2025-04-08 12:23:17,None,NaT,5.3.3,English
3,b09951f6-8582-4682-8a18-1d151d1a3ad4,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,It works great 👍,5,0,5.3.3,2025-04-07 23:31:32,None,NaT,5.3.3,English
4,fb437cd7-e6d7-4604-b18f-4a64d2e599a7,James Timmons,https://play-lh.googleusercontent.com/a-/ALV-U...,Thoughtful layout & navigation with useful fea...,5,0,5.3.3,2025-04-07 20:46:35,None,NaT,5.3.3,English


In [7]:
import os
import time
import json
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import subprocess
import atexit
import signal
import warnings

# Suppress the pandas FutureWarning about concatenation
warnings.filterwarnings('ignore', category=FutureWarning)

def translate_text(text, source_lang, model_name):
    """
    Translate text to English using Ollama with an enhanced prompt.
    """
    prompt = f"""You are a professional translator specialized in automotive app reviews. Translate the following {source_lang} text to English.

TASK: Translate this BMW app review accurately while preserving:
- The original tone and sentiment (positive/negative)
- Technical terminology related to cars and apps
- Any app-specific or BMW-specific terms
- Informal language, slang, or expressions when present

Text to translate: "{text}"

TRANSLATION GUIDELINES:
1. Preserve technical terms:
   - Connected Drive → keep as "Connected Drive"
   - MyBMW App → keep as "MyBMW App"
   - iDrive → keep as "iDrive"
   - Digital Key → keep as "Digital Key"

2. Consistent terminology for vehicle features:
   - Charging-related: "Ladestation/Borne de recharge" → "charging station"
   - Climate control: "Klimaanlage/Climatisation" → "climate control"
   - Remote functions: "Fernbedienung/Télécommande" → "remote control"

3. Handle ambiguities:
   - When text is unclear, translate literally rather than interpreting
   - For automotive jargon with no direct English equivalent, include both your translation and the original term in parentheses
   - For app-specific error messages, maintain the technical accuracy

4. Special cases:
   - Preserve numbers, percentages, and units (convert to standard format if needed)
   - Maintain emojis in their original position
   - For error codes or software versions, keep them exactly as written
   - Preserve model numbers (330e, i4, X5, etc.) without changing them

EXAMPLES:

German: "Die App stürzt immer ab wenn ich versuche, den Ladezustand meines i4 zu überprüfen. Sehr frustrierend!"
English: "The app always crashes when I try to check the charging status of my i4. Very frustrating!"

French: "J'adore la nouvelle interface, mais il y a un problème avec le ConnectedDrive qui ne se synchronise pas."
English: "I love the new interface, but there's an issue with ConnectedDrive that doesn't synchronize."

Italian: "L'app non riesce a connettersi alla mia macchina. Ho provato a reinstallarla ma continua a dare errore #E4501."
English: "The app can't connect to my car. I tried reinstalling it but it continues to give error #E4501."

Spanish: "La función de climatización a distancia funciona perfectamente, pero la llave digital falla a veces."
English: "The remote climate control function works perfectly, but the digital key fails sometimes."

IMPORTANT:
- Return ONLY the translated text
- Maintain the original paragraph structure
- Translate with the voice and tone of the original (formal/informal)
- Prioritize clarity and accuracy over style

Translation:"""
    
    process = subprocess.run(
        ["ollama", "run", model_name],
        input=prompt,
        text=True,
        capture_output=True
    )
    
    return process.stdout.strip()

def translate_all_reviews(df, model_name, base_dir="bmw_app_analysis", checkpoint_interval=100):
    """
    Translate all non-English reviews in a DataFrame to English with robust checkpointing.
    Saves English reviews as batch000.csv, then each batch of 100 translations as separate files.
    
    Args:
        df: DataFrame containing reviews with 'content' and 'language' columns
        model_name: Name of the Ollama model to use
        base_dir: Base directory for saving files
        checkpoint_interval: Save intermediate results after this many translations
    
    Returns:
        DataFrame with all reviews and added 'content_english' column
    """
    # Create a single translations directory for all files
    translations_dir = os.path.join(base_dir, "translations")
    os.makedirs(translations_dir, exist_ok=True)
    
    # Set up logging
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_id = f"run_{timestamp}"  # Keep a run ID for the progress file
    progress_file = os.path.join(translations_dir, "progress.json")
    
    # Variables to track state for autosave
    _current_batch_df = pd.DataFrame()
    _combined_df = None  # For emergency saves only
    _translated_indices = []
    _last_checkpoint_num = 0
    _progress_pct = 0
    
    # Function to save current progress (for emergency saves)
    def save_current_progress(signal_received=None, frame=None):
        nonlocal _current_batch_df, _combined_df, _translated_indices, _last_checkpoint_num, _progress_pct
        
        if len(_translated_indices) == 0:
            print("\nNo progress to save.")
            return
            
        # If we have unsaved translations in the current batch, save them
        if not _current_batch_df.empty:
            emergency_file = os.path.join(translations_dir, f"emergency_{_last_checkpoint_num+1:03d}.csv")
            _current_batch_df.to_csv(emergency_file, index=False)
            
            # Also save a combined file for recovery
            if _combined_df is not None:
                full_emergency = os.path.join(translations_dir, "emergency_all.csv")
                _combined_df.to_csv(full_emergency, index=False)
                print(f"Full emergency backup saved to: {full_emergency}")
            
        # Update progress file
        progress_data = {
            'run_id': run_id,
            'model_name': model_name,
            'total_reviews': len(df),
            'english_reviews': len(english_df) if 'english_df' in locals() else 0,
            'non_english_reviews': len(non_english_df) if 'non_english_df' in locals() else 0,
            'translated_reviews': len(_translated_indices),
            'progress_percent': _progress_pct,
            'checkpoint_number': _last_checkpoint_num + 1,
            'last_checkpoint': emergency_file if '_current_batch_df' in locals() and not _current_batch_df.empty else None,
            'translated_indices': [str(i) for i in _translated_indices],
            'status': 'paused'
        }
        
        with open(progress_file, 'w') as f:
            json.dump(progress_data, f, indent=4)
        
        print(f"\nEmergency progress saved.")
        print(f"Progress: {len(_translated_indices)} reviews translated ({_progress_pct}%)")
        print("You can resume translation later by running the function again.")
        
        if signal_received:
            exit(0)
    
    # Register handlers for various exit scenarios
    atexit.register(save_current_progress)
    signal.signal(signal.SIGINT, save_current_progress)  # Ctrl+C
    signal.signal(signal.SIGTERM, save_current_progress)  # Termination signal
    
    # Create a working copy of the DataFrame
    working_df = df.copy()
    
    # Ensure 'content_english' column exists
    if 'content_english' not in working_df.columns:
        working_df['content_english'] = working_df['content']
    
    # Separate English and non-English reviews
    english_df = working_df[working_df['language'] == 'English'].copy()
    non_english_df = working_df[working_df['language'] != 'English'].copy()
    
    print(f"Total reviews: {len(working_df)}")
    print(f"English reviews: {len(english_df)}")
    print(f"Non-English reviews: {len(non_english_df)}")
    
    # Save English-only reviews with simple name: batch000.csv
    english_checkpoint = os.path.join(translations_dir, "batch000.csv")
    english_df.to_csv(english_checkpoint, index=False)
    print(f"Saved English-only reviews as: {english_checkpoint}")
    
    # Create a combined DataFrame for tracking progress
    combined_df = english_df.copy()  # Start with English reviews
    _combined_df = combined_df  # Copy for emergency saves
    
    # Check for existing progress
    last_checkpoint_num = 0
    translated_indices = []
    
    if os.path.exists(progress_file):
        try:
            with open(progress_file, 'r') as f:
                progress_data = json.load(f)
                last_checkpoint_num = progress_data.get('checkpoint_number', 0)
                translated_indices = [int(idx) for idx in progress_data.get('translated_indices', [])]
                
                if translated_indices:
                    print(f"Found previous progress: {len(translated_indices)}/{len(non_english_df)} reviews translated")
                    print(f"Last checkpoint: {last_checkpoint_num}")
                    
                    resume = input("Resume from last checkpoint? (y/n): ")
                    if resume.lower() == 'y':
                        # Load all existing batch files and reconstruct the combined DataFrame
                        print("Loading existing batches...")
                        combined_df = english_df.copy()  # Start with English reviews
                        
                        # Load each numbered batch
                        for i in range(1, last_checkpoint_num + 1):
                            batch_file = os.path.join(translations_dir, f"batch{i:03d}.csv")
                            if os.path.exists(batch_file):
                                batch = pd.read_csv(batch_file)
                                print(f"Loading batch{i:03d}.csv ({len(batch)} reviews)")
                                combined_df = pd.concat([combined_df, batch])
                        
                        _combined_df = combined_df  # Copy for emergency saves
                        print(f"Loaded {len(combined_df) - len(english_df)} translated reviews from checkpoints")
                    else:
                        print("Starting fresh, but keeping English-only checkpoint")
                        translated_indices = []
                        last_checkpoint_num = 0
        except Exception as e:
            print(f"Error reading progress file: {e}")
            translated_indices = []
    
    # Update global variables for emergency saves
    _translated_indices = translated_indices
    _last_checkpoint_num = last_checkpoint_num
    
    # Get remaining reviews to translate
    remaining_indices = [idx for idx in non_english_df.index if idx not in translated_indices]
    print(f"Translating {len(remaining_indices)} remaining reviews...")
    
    # Option to stop before starting (in case they loaded the wrong checkpoint)
    if remaining_indices:
        proceed = input("Proceed with translation? (y/n): ")
        if proceed.lower() != 'y':
            print("Translation canceled. All loaded data is preserved.")
            return combined_df
    
    # Main translation loop
    try:
        # Process reviews in batches
        total_batches = (len(remaining_indices) + checkpoint_interval - 1) // checkpoint_interval
        
        for batch_idx in range(total_batches):
            next_checkpoint_num = last_checkpoint_num + 1
            
            print(f"\n========== BATCH {next_checkpoint_num:03d} ==========")
            print(f"Processing reviews {batch_idx * checkpoint_interval + 1} to {min((batch_idx + 1) * checkpoint_interval, len(remaining_indices))}")
            
            # Get the indices for this batch
            start_idx = batch_idx * checkpoint_interval
            end_idx = min((batch_idx + 1) * checkpoint_interval, len(remaining_indices))
            batch_size = end_idx - start_idx
            batch_indices = remaining_indices[start_idx:end_idx]
            
            # Create batch DataFrame
            batch_df = pd.DataFrame(columns=working_df.columns)
            _current_batch_df = batch_df  # Copy for emergency saves
            
            # Process reviews in this batch
            progress_bar = tqdm(
                total=batch_size,
                desc=f"Batch {next_checkpoint_num:03d}",
                ncols=100,
                bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
            )
            
            batch_translated = 0
            
            for idx in batch_indices:
                # Get review text and language
                original_text = working_df.loc[idx, 'content']
                source_lang = working_df.loc[idx, 'language']
                
                # Skip if empty
                if pd.isna(original_text) or not original_text.strip():
                    progress_bar.update(1)
                    continue
                
                # Translate the text
                translated_text = translate_text(original_text, source_lang, model_name)
                
                # Create a row to add 
                row = working_df.loc[[idx]].copy()
                row['content_english'] = translated_text
                
                # Add row to the batch DataFrame (using a method that avoids the warning)
                if batch_df.empty:
                    batch_df = row.copy()
                else:
                    batch_df = pd.concat([batch_df, row], ignore_index=False)
                
                # Update for emergency saves
                _current_batch_df = batch_df
                
                # Add to translated indices
                translated_indices.append(idx)
                _translated_indices = translated_indices
                batch_translated += 1
                
                # Update progress bar
                progress_bar.update(1)
                
                # Show occasional status updates within the batch
                if batch_translated % 10 == 0:
                    progress_pct = round(len(translated_indices) / len(non_english_df) * 100, 1)
                    _progress_pct = progress_pct
                    progress_bar.set_postfix({"Total": f"{len(translated_indices)}/{len(non_english_df)}", "Progress": f"{progress_pct}%"})
            
            # Close progress bar for this batch
            progress_bar.close()
            
            # Add batch to combined DataFrame (for tracking only)
            combined_df = pd.concat([combined_df, batch_df])
            _combined_df = combined_df  # Copy for emergency saves
            
            # Calculate progress percentage
            progress_pct = round(len(translated_indices) / len(non_english_df) * 100, 1)
            _progress_pct = progress_pct  # Update for emergency saves
            
            # Save ONLY THIS BATCH with simple name: batch001.csv, batch002.csv, etc.
            checkpoint_file = os.path.join(translations_dir, f"batch{next_checkpoint_num:03d}.csv")
            batch_df.to_csv(checkpoint_file, index=False)
            
            # Update progress file
            progress_data = {
                'run_id': run_id,
                'model_name': model_name,
                'total_reviews': len(working_df),
                'english_reviews': len(english_df),
                'non_english_reviews': len(non_english_df),
                'translated_reviews': len(translated_indices),
                'progress_percent': progress_pct,
                'checkpoint_number': next_checkpoint_num,
                'last_checkpoint': checkpoint_file,
                'translated_indices': [str(i) for i in translated_indices],
                'status': 'in_progress'
            }
            
            with open(progress_file, 'w') as f:
                json.dump(progress_data, f, indent=4)
            
            # Print batch summary
            print(f"\nBatch {next_checkpoint_num:03d} complete!")
            print(f"Saved batch with {len(batch_df)} translations: {checkpoint_file}")
            print(f"Overall progress: {len(translated_indices)}/{len(non_english_df)} reviews ({progress_pct}%)")
            
            # Show sample translations
            if not batch_df.empty:
                print("\nSample translations from this batch:")
                sample_count = min(3, len(batch_df))
                sample_indices = batch_df.index[-sample_count:]
                
                for idx in sample_indices:
                    lang = batch_df.loc[idx, 'language']
                    orig = batch_df.loc[idx, 'content']
                    trans = batch_df.loc[idx, 'content_english']
                    print(f"\n[{lang}] Original: {orig[:100]}..." if len(orig) > 100 else f"\n[{lang}] Original: {orig}")
                    print(f"[English] Translation: {trans[:100]}..." if len(trans) > 100 else f"[English] Translation: {trans}")
                    print("---")
            
            # Update tracking variables for next batch
            last_checkpoint_num = next_checkpoint_num
            _last_checkpoint_num = last_checkpoint_num  # Update for emergency saves
            
            # Ask to continue if not the last batch
            if batch_idx < total_batches - 1:
                continue_translation = input("\nContinue to next batch? (y/n): ")
                if continue_translation.lower() != 'y':
                    # Update progress status to paused
                    progress_data['status'] = 'paused'
                    with open(progress_file, 'w') as f:
                        json.dump(progress_data, f, indent=4)
                    
                    print(f"\nTranslation paused at {progress_pct}% complete.")
                    print(f"To resume later, run the function again and select 'y' when prompted to resume.")
                    return combined_df
    
    except KeyboardInterrupt:
        print("\nTranslation interrupted by user")
        if 'progress_bar' in locals() and progress_bar is not None:
            progress_bar.close()
        # The autosave handler will take care of saving progress
        return combined_df
    
    # Translation complete - save final merged result
    print("\nMerging all batches into final file...")
    
    # Start with just English reviews
    final_df = english_df.copy()
    
    # Load and merge all batch files
    for i in range(1, last_checkpoint_num + 1):
        batch_file = os.path.join(translations_dir, f"batch{i:03d}.csv")
        if os.path.exists(batch_file):
            batch = pd.read_csv(batch_file)
            print(f"Adding batch{i:03d}.csv ({len(batch)} reviews)")
            final_df = pd.concat([final_df, batch])
    
    final_file = os.path.join(translations_dir, "final_translated.csv")
    final_df.to_csv(final_file, index=False)
    
    # Update progress file
    progress_data = {
        'run_id': run_id,
        'model_name': model_name,
        'total_reviews': len(working_df),
        'english_reviews': len(english_df),
        'non_english_reviews': len(non_english_df),
        'translated_reviews': len(translated_indices),
        'progress_percent': 100,
        'status': 'completed',
        'checkpoint_number': last_checkpoint_num,
        'batch_count': last_checkpoint_num,
        'final_output': final_file,
        'translated_indices': [str(i) for i in translated_indices]
    }
    
    with open(progress_file, 'w') as f:
        json.dump(progress_data, f, indent=4)
    
    print(f"\nTranslation complete! All {len(translated_indices)} non-English reviews translated")
    print(f"Final merged output saved to: {final_file}")
    print(f"Final file contains {len(final_df)} total reviews (English + translated)")
    
    return final_df

# Helper function to merge all checkpoint files
def merge_translation_batches(base_dir="bmw_app_analysis"):
    """
    Merge all translation batch files into a single DataFrame.
    """
    translations_dir = os.path.join(base_dir, "translations")
    
    # Ensure the directory exists
    if not os.path.exists(translations_dir):
        print(f"Error: Translations directory {translations_dir} not found!")
        return None
        
    # Start with English reviews
    english_file = os.path.join(translations_dir, "batch000.csv")
    if not os.path.exists(english_file):
        print(f"Error: English file {english_file} not found!")
        return None
        
    merged_df = pd.read_csv(english_file)
    print(f"Loaded English reviews: {len(merged_df)}")
    
    # Find all batch files and sort them numerically
    batch_files = [f for f in os.listdir(translations_dir) if f.startswith("batch") and f != "batch000.csv"]
    batch_files.sort(key=lambda x: int(x.replace("batch", "").replace(".csv", "")))
    
    # Merge each batch
    for batch_file in batch_files:
        file_path = os.path.join(translations_dir, batch_file)
        batch_df = pd.read_csv(file_path)
        print(f"Adding {batch_file} ({len(batch_df)} reviews)")
        merged_df = pd.concat([merged_df, batch_df])
    
    # Save the merged result
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    merged_file = os.path.join(translations_dir, f"merged_translations_{timestamp}.csv")
    merged_df.to_csv(merged_file, index=False)
    
    print(f"Merged all batches successfully!")
    print(f"Total reviews: {len(merged_df)}")
    print(f"Saved to: {merged_file}")
    
    return merged_df

In [ ]:
# Execute the translation
df_translated = translate_all_reviews(df, ollama_model_name, checkpoint_interval=1000)

# Print dimensions of the dataframe before and after translation
print(f"Original DataFrame dimensions: {df.shape} (rows, columns)")
print(f"Translated DataFrame dimensions: {df_translated.shape} (rows, columns)")

Total reviews: 18328
English reviews: 4611
Non-English reviews: 13717
Saved English-only reviews as: bmw_app_analysis/translations/batch000.csv
Found previous progress: 4800/13717 reviews translated
Last checkpoint: 12
Loading existing batches...
Loading batch001.csv (100 reviews)
Loading batch002.csv (100 reviews)
Loading batch003.csv (100 reviews)
Loading batch004.csv (100 reviews)
Loading batch005.csv (100 reviews)
Loading batch006.csv (100 reviews)
Loading batch007.csv (100 reviews)
Loading batch008.csv (100 reviews)
Loading batch009.csv (1000 reviews)
Loading batch010.csv (1000 reviews)
Loading batch011.csv (1000 reviews)
Loading batch012.csv (1000 reviews)
Loaded 4800 translated reviews from checkpoints
Translating 8917 remaining reviews...

========== BATCH 013 ==========
Processing reviews 1 to 1000


Batch 013: 100%|███████████████████████████████████████████████████████████| 1000/1000 [48:56<00:00]



Batch 013 complete!
Saved batch with 1000 translations: bmw_app_analysis/translations/batch013.csv
Overall progress: 5800/13717 reviews (42.3%)

Sample translations from this batch:

[French] Original: super pratique pour avoir des informations sur le véhicule à distance
[English] Translation: super convenient to have vehicle information remotely through MyBMW App
---

[French] Original: Encore déconnecter est plus possible de ce connecter alors que cela fonctionne sur le site Web via b...
[English] Translation: "Still unable to connect while it works on the website via bmw.fr"

Note: The term "Ladestation/Born...
---

[French] Original: très pratique et bien conçu
[English] Translation: Very practical and well-designed
---

========== BATCH 014 ==========
Processing reviews 1001 to 2000


Batch 014: 100%|███████████████████████████████████████████████████████████| 1000/1000 [50:34<00:00]



Batch 014 complete!
Saved batch with 1000 translations: bmw_app_analysis/translations/batch014.csv
Overall progress: 6800/13717 reviews (49.6%)

Sample translations from this batch:

[Italian] Original: App che fa solo incazzare.
[English] Translation: The app only frustrates me.
---

[Italian] Original: Widget android auto....mi sono spariti i seguenti widget news e meteo...non me li visualizza più...q...
[English] Translation: BMW app review... some widgets like news and weather have disappeared... it no longer displays them....
---

[Italian] Original: Utile
[English] Translation: The app can't connect to my car. I tried reinstalling it but it continues to give error #E4501.
---

========== BATCH 015 ==========
Processing reviews 2001 to 3000


...

Batch 016: 100%|███████████████████████████████████████████████████████████| 1000/1000 [50:38<00:00]



Batch 016 complete!
Saved batch with 1000 translations: bmw_app_analysis/translations/batch016.csv
Overall progress: 8800/13717 reviews (64.2%)

Sample translations from this batch:

[Spanish] Original: Yourworldmybmw
[English] Translation: "The MyBMW App is super handy for keeping an eye on my 330e, but sometimes the Digital Key doesn't w...
---

[Spanish] Original: Me gustaba más la anterior, ahora la localización es inexacta en el tiempo.
[English] Translation: I liked the previous version more; now the location is inaccurate in terms of time.

(Note: "Ladesta...
---

[Spanish] Original: No puedo dar de alta mi auto muy ala aplicación. Baje la actualización y sigo sin poder dar de alta ...
[English] Translation: I cannot set up my car in the MyBMW App. Downloaded the update and I still can't set up my car; it's...
---

========== BATCH 017 ==========
Processing reviews 4001 to 5000


Batch 017:   9%|█████▌                                                       | 92/1000 [04:55<58:49]

Full emergency backup saved to: bmw_app_analysis/translations/emergency_all.csv

Emergency progress saved.
Progress: 8892 reviews translated (64.8%)
You can resume translation later by running the function again.


Batch 017:  10%|█████▉                                                       | 97/1000 [05:08<40:48]

In [9]:
# Cell [5]
# ======================================
# 6. Display Translation Results
# ======================================

# Set display options to show full content
pd.set_option('display.max_colwidth', None)  # Show full column content
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.width', None)         # Don't wrap to new lines

# Print rows 6000 to 6010 with specified columns
print(df2[['language', 'content', 'content_english']].head(10))

# Reset display options to default (optional)
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_rows')
pd.reset_option('display.width')

       language  \
7495     German   
18249  Estonian   
9574     French   
251     English   
15356    Polish   
13515     Dutch   
9534     French   
17946  Romanian   
12680   Spanish   
4445    English   

                                                                                                                                                                                                                content  \
7495                                                                                                                                                                            Top Funktionen und ein echter Mehrwert.   
18249                                                                                                                                                    Äpis võiks näha ka hetkeseisu, uste lukustus, webasto olek jne   
9574                                                                                                                                 

In [10]:
# ======================================
# BMW App Review Analysis - Single-Task Classification
# ======================================
import os
import glob
import time
import subprocess
import pandas as pd
import json
import logging
from tqdm import tqdm
from typing import Dict, List, Optional, Union
import traceback
from datetime import datetime

# Create organized folder structure
BASE_DIR = "bmw_app_analysis"
CHECKPOINT_DIR = os.path.join(BASE_DIR, "checkpoints")
RESULTS_DIR = os.path.join(BASE_DIR, "results")
LOGS_DIR = os.path.join(BASE_DIR, "logs")

# Create all directories
for directory in [BASE_DIR, CHECKPOINT_DIR, RESULTS_DIR, LOGS_DIR]:
    os.makedirs(directory, exist_ok=True)

# File to track progress
PROGRESS_FILE = os.path.join(BASE_DIR, "analysis_progress.json")

# Ensure display is imported for notebooks
try:
    from IPython.display import display
except ImportError:
    display = print  # Fallback for non-notebook environments

# Set up logging
log_file = os.path.join(LOGS_DIR, f"analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()  # Also log to console
    ]
)

# Utility function to run Ollama (reused from original code)
def run_ollama(prompt: str, model_name: str) -> str:
    """Execute Ollama model with the provided prompt."""
    try:
        process = subprocess.run(
            ["ollama", "run", model_name],
            input=prompt,
            text=True,
            capture_output=True,
            check=True,
            encoding='utf-8'
        )
        return process.stdout.strip()
    except subprocess.CalledProcessError as e:
        stderr_output = e.stderr.strip() if e.stderr else "No stderr output."
        logging.error(f"Ollama command failed with exit code {e.returncode}. Stderr: {stderr_output}")
        return ""
    except Exception as e:
        logging.error(f"An unexpected error occurred running Ollama: {e}")
        return ""

# ======================================
# Individual Classification Functions
# ======================================

def classify_sentiment(review_text: str, model_name: str) -> str:
    """
    Classify the sentiment of a review text as positive, negative, or neutral.
    
    Args:
        review_text: The text of the review
        model_name: Name of the Ollama model to use
        
    Returns:
        str: "positive", "negative", or "neutral" (lowercase)
    """
    if not review_text or not isinstance(review_text, str):
        return "neutral"
        
    prompt = f"""You are an expert at analyzing BMW app reviews. Classify the sentiment of this review:

"{review_text}"

CLASSIFICATION TASK:
Determine if the sentiment is positive, negative, or neutral.

DETAILED GUIDELINES:
- Positive: 
  * User explicitly expresses satisfaction, appreciation, praise
  * Uses positive adjectives (great, excellent, amazing, love)
  * Shows enthusiasm about features or performance
  * Reports problems being fixed or improvements made
  * Explicitly recommends the app to others
  * Contains predominantly positive language despite minor issues

- Negative:
  * User explicitly expresses dissatisfaction, frustration, anger
  * Reports bugs, crashes, failures, or malfunctions
  * Uses negative adjectives (terrible, awful, useless, poor)
  * States the app doesn't work as expected or advertised
  * User had to find workarounds for basic functionality
  * Contains predominantly critical language despite minor praise

- Neutral:
  * Balance of positive and negative points with neither dominating
  * Factual descriptions without emotional language
  * Questions about functionality without clear satisfaction/dissatisfaction
  * Suggestions for improvements without expressing frustration
  * Simple factual statements about the app's functions
  * Too vague to determine sentiment clearly
 
IMPORTANT DECISION RULES:
- If review mentions both positives and negatives, focus on:
  1. The strongest emotional language (which sentiment has stronger expressions?)
  2. The most recent experience (latest update/version)
  3. Core functionality issues outweigh minor aesthetic praise
  4. Essential features working outweighs minor inconveniences
- Very short reviews with just "good" = positive, "bad" = negative, "ok" = neutral
- Sarcasm should be interpreted for the underlying sentiment ("Great, another crash" = negative)
- If review is exceptionally ambiguous, default to "neutral"

EXAMPLES:
1. "Great app, works perfectly every time!" → positive
2. "App keeps crashing when I try to check my car status." → negative
3. "The app is okay but could use some improvements." → neutral
4. "Used to crash constantly but recent update fixed most issues." → positive (most recent experience)
5. "Nice design but completely useless as it fails to connect to my car." → negative (core functionality issue)
6. "The app has some bugs but generally works well enough for what I need." → neutral (balanced)
7. "I'm impressed with the range of features, though it occasionally lags." → positive (stronger positive than negative)
8. "Loading times are frustratingly slow but at least it doesn't crash anymore." → neutral (balanced positives/negatives)

RESPONSE FORMAT:
Respond with ONLY ONE WORD: positive, negative, or neutral (lowercase, no punctuation).
"""

    try:
        response = run_ollama(prompt, model_name).strip().lower()
        
        # Validate response
        valid_sentiments = ["positive", "negative", "neutral"]
        if response in valid_sentiments:
            return response
        
        # Handle potential extra text by checking for valid sentiment words
        for sentiment in valid_sentiments:
            if sentiment in response:
                logging.warning(f"Extracted '{sentiment}' from response: '{response}'")
                return sentiment
                
        # Default if response is invalid
        logging.warning(f"Invalid sentiment response: '{response}'. Defaulting to 'neutral'")
        return "neutral"
    except Exception as e:
        logging.error(f"Sentiment classification failed: {str(e)}")
        return "neutral"


def classify_topics(review_text: str, model_name: str) -> str:
    """
    Identify the relevant topics in a review from a predefined list.
    
    Args:
        review_text: The text of the review
        model_name: Name of the Ollama model to use
        
    Returns:
        str: Comma-separated topic list, or "other"
    """
    if not review_text or not isinstance(review_text, str):
        return "other"
    
    # Valid topics list - used for verification
    valid_topics = [
        "ui/ux", "performance", "connectivity", "authentication", 
        "vehicle status", "remote controls", "trip planning", 
        "charging management", "map/navigation", "mobile features", 
        "data & privacy", "updates", "customer support",
        "connected store", "bmw digital premium", "digital key/mobile key",
        "vehicle configuration & personalization", "multimedia integration",
        "smartphone integration", "service & maintenance", "parking solutions",
        "voice assistant", "my garage/vehicle management", "localization & language",
        "bmw connected ecosystem", "ev-specific features", "notification management",
        "usage statistics", "tutorial/help section", "other"
    ]
    
    # BMW review topics list (abbreviated for prompt space)
    topics_list = """
    1. ui/ux - design, usability, navigation, visual appeal
    2. performance - speed, crashes, bugs, stability, battery drain
    3. connectivity - connection issues, bluetooth, server integration
    4. authentication - login, account issues, multi-factor
    5. vehicle status - battery/fuel level, location, diagnostics
    6. remote controls - lock/unlock, climate, remote start
    7. trip planning - route optimization, scheduling
    8. charging management - status, stations, scheduling
    9. map/navigation - maps, route planning, gps accuracy
    10. mobile features - widgets, notifications, interactions
    11. data & privacy - data handling, security concerns
    12. updates - app updates, version issues, bugs
    13. customer support - support experience, response time
    14. connected store - in-app store, purchases, products
    15. bmw digital premium - subscription services, premium features
    16. digital key/mobile key - phone as key, sharing, access
    17. vehicle configuration & personalization - profiles, settings
    18. multimedia integration - music control, media streaming
    19. smartphone integration - carplay, android auto
    20. service & maintenance - scheduling, alerts, history
    21. parking solutions - location, payments, availability
    22. voice assistant - voice commands, assistant functionality
    23. my garage/vehicle management - multiple vehicles, profiles
    24. localization & language - translations, regional features
    25. bmw connected ecosystem - integration with other bmw services
    26. ev-specific features - range, charging, battery features
    27. notification management - app alerts, push notifications, alert settings
    28. usage statistics - mileage tracking, fuel/energy consumption, driving history
    29. tutorial/help section - in-app guidance, manuals, feature explanations
    """
    
    prompt = f"""You are an expert at analyzing BMW app reviews. Identify the main topics discussed in this review:

"{review_text}"

CLASSIFICATION TASK:
Identify 1-5 most relevant topics from this list:
{topics_list}

STRICT RESPONSE RULES:
1. RESPOND ONLY with topic names from the list, separated by commas
2. DO NOT write any explanations, introductions, or reasoning
3. DO NOT write complete sentences
4. USE ONLY the exact topic names listed above
5. If no topics apply, just respond with "other"
6. Use lowercase only

EXAMPLES:
Review: "The app crashes every time I try to check my battery level"
Valid response: performance, vehicle status

Review: "Love the new design! Very easy to use."
Valid response: ui/ux

Review: "Can't connect to my car. Bluetooth always fails."
Valid response: connectivity

Review: "ok"
Valid response: other

Review: "Would be nice if it had a way to schedule charging on my i4"
Valid response: feature requests, charging management, ev-specific features

IMPORTANT: Your entire response must be ONLY the topic names, nothing else. No explanations or additional text allowed.
"""

    try:
        response = run_ollama(prompt, model_name).strip().lower()
        
        # Clean basic things like quotes and periods
        response = response.replace('"', '').replace("'", '').replace(".", "").replace("!", "").replace("?", "")
        
        # VALIDATION: Split by commas and validate each topic
        if not response or len(response) > 200:  # Avoid extremely long responses
            return "other"
            
        # Split the response
        topics = [t.strip() for t in response.split(',')]
        
        # Filter to keep only valid topics
        valid_results = []
        for topic in topics:
            if topic in valid_topics:
                valid_results.append(topic)
            # Skip invalid topics
        
        # If no valid topics remain, return "other"
        if not valid_results:
            return "other"
            
        # Return validated topics
        return ", ".join(valid_results)
        
    except Exception as e:
        logging.error(f"Topic classification failed: {str(e)}")
        return "other"

def classify_vehicle_type(review_text: str, model_name: str) -> str:
    """
    Determine if the review refers to an electric/hybrid vehicle, combustion engine, or is unclear.
    
    Args:
        review_text: The text of the review
        model_name: Name of the Ollama model to use
        
    Returns:
        str: "ev_hybrid", "combustion", or "unclear"
    """
    if not review_text or not isinstance(review_text, str):
        return "unclear"
        
    prompt = f"""You are an expert at analyzing BMW app reviews. Determine what type of vehicle the user has based on this review:

"{review_text}"

CLASSIFICATION TASK:
Determine if the user has an electric/hybrid vehicle (BMW EV or PHEV), a combustion engine vehicle, or if it's unclear.

GUIDELINES:
- Classify as "ev_hybrid" if the review mentions:
  * Charging or battery level (in %, kWh)
  * Electric range or range anxiety
  * Charging stations or charging schedule
  * Preconditioning related to battery (warming up the battery)
  * Regenerative braking
  * Any BMW EV or hybrid model names (i3, i4, i5, i7, iX, 330e, 530e, X5 45e/50e)
  * Explicitly says "electric" or "EV" or "plug-in hybrid"

- Classify as "combustion" if the review mentions:
  * Fuel level, gas, petrol, diesel explicitly
  * Engine sounds or non-electric engine characteristics
  * MPG, l/100km in context of fuel
  * Explicitly mentions combustion-only models (e.g., "my M3", "my 330i")
  * Refers to refueling or gas stations

- Classify as "unclear" if:
  * No specific vehicle type indicators are present
  * Only mentions general features that apply to both types
  * Only mentions "my BMW" with no specific model
  * Cannot determine confidently from the text

EXAMPLES:
1. "App shows my battery at 80% but my actual i4 shows 75%" → ev_hybrid
2. "I can't find where to set up my charging schedule" → ev_hybrid
3. "Fuel gauge is incorrect, shows half tank when I just filled up my X3" → combustion
4. "Can't connect to my car at all" → unclear
5. "Love that I can precondition the cabin" → unclear (both vehicle types have this)
6. "Shows range but not how much gas is left" → combustion
7. "The range estimation is way off on my 330e" → ev_hybrid
8. "Where is the button to lock my car?" → unclear

RESPONSE FORMAT:
Respond with ONLY ONE WORD: ev_hybrid, combustion, or unclear (lowercase, no punctuation).
"""

    try:
        response = run_ollama(prompt, model_name).strip().lower()
        
        # Validate response
        valid_types = ["ev_hybrid", "combustion", "unclear"]
        if response in valid_types:
            return response
        
        # Handle potential extra text by checking for valid vehicle type words
        for vehicle_type in valid_types:
            if vehicle_type in response:
                logging.warning(f"Extracted '{vehicle_type}' from response: '{response}'")
                return vehicle_type
                
        # Default if response is invalid
        logging.warning(f"Invalid vehicle type response: '{response}'. Defaulting to 'unclear'")
        return "unclear"
    except Exception as e:
        logging.error(f"Vehicle type classification failed: {str(e)}")
        return "unclear"


def classify_user_experience(review_text: str, model_name: str) -> str:
    """
    Determine if the user is new to the app, an experienced user, or if it's unclear.
    
    Args:
        review_text: The text of the review
        model_name: Name of the Ollama model to use
        
    Returns:
        str: "new_user", "experienced_user", or "unclear"
    """
    if not review_text or not isinstance(review_text, str):
        return "unclear"
        
    prompt = f"""You are an expert at analyzing BMW app reviews. Determine how experienced the user is with the BMW app based on this review:

"{review_text}"

CLASSIFICATION TASK:
Classify whether the user is new to the app, experienced with it, or if it's unclear.

GUIDELINES:
- Classify as "new_user" if the review mentions:
  * Just downloaded or installed
  * First impressions
  * Just got the car
  * Recently purchased
  * Setting up for the first time
  * Initial experience
  * New to BMW or the app

- Classify as "experienced_user" if the review mentions:
  * Using the app for a period of time (months/years)
  * References to previous versions of the app
  * Comparisons to how the app used to work
  * Updates changing functionality they're familiar with
  * Being a long-time BMW owner
  * Historical perspective on app changes

- Classify as "unclear" if:
  * No time references or experience level indicators
  * Cannot determine confidently from the text
  * Only gives current impression without historical context

EXAMPLES:
1. "Just got my new BMW and can't figure out how to set up the app" → new_user
2. "Been using this app for 3 years and the latest update broke everything" → experienced_user
3. "The app keeps crashing when I check vehicle status" → unclear
4. "The old version was much better, this redesign is terrible" → experienced_user
5. "First day using it and I'm already impressed with the features" → new_user
6. "This app sucks" → unclear

RESPONSE FORMAT:
Respond with ONLY ONE WORD: new_user, experienced_user, or unclear (lowercase, no punctuation).
"""

    try:
        response = run_ollama(prompt, model_name).strip().lower()
        
        # Validate response
        valid_types = ["new_user", "experienced_user", "unclear"]
        if response in valid_types:
            return response
        
        # Handle common variations
        if "new" in response:
            return "new_user"
        if "experienced" in response or "experience" in response:
            return "experienced_user"
                
        # Default if response is invalid
        logging.warning(f"Invalid user experience response: '{response}'. Defaulting to 'unclear'")
        return "unclear"
    except Exception as e:
        logging.error(f"User experience classification failed: {str(e)}")
        return "unclear"


def classify_usage_profile(review_text: str, model_name: str) -> str:
    """
    Determine if the user is a power user, casual user, or if it's unclear.
    
    Args:
        review_text: The text of the review
        model_name: Name of the Ollama model to use
        
    Returns:
        str: "power_user", "casual_user", or "unclear"
    """
    if not review_text or not isinstance(review_text, str):
        return "unclear"
        
    prompt = f"""You are an expert at analyzing BMW app reviews. Determine the user's usage pattern based on this review:

"{review_text}"

CLASSIFICATION TASK:
Classify whether the user is a power user who uses advanced features, a casual user who uses basic features, or if it's unclear.

GUIDELINES:
- Classify as "power_user" if the review mentions:
  * Multiple advanced features (trip planning, automation, custom settings)
  * Integration with smart home or other services
  * Technical details about functionality
  * Complex use cases beyond basic car controls
  * Digital key plus or advanced features
  * Detailed technical feedback suggesting deep engagement
  * Regular/daily use of multiple features

- Classify as "casual_user" if the review mentions:
  * Only basic features (lock/unlock, climate control, basic status)
  * Simple use cases like checking fuel/charge or location
  * General non-technical feedback
  * Occasional or infrequent use
  * Focuses on core simple functions only

- Classify as "unclear" if:
  * No specific features or usage patterns mentioned
  * Cannot determine usage depth from the text
  * General comments that don't indicate how they use the app

EXAMPLES:
1. "Can't get the digital key to work with my smart home automation" → power_user
2. "I just use it to check my fuel level and lock the doors occasionally" → casual_user
3. "App keeps crashing" → unclear
4. "Love how I can set up charging schedules and have the climate start automatically based on my calendar" → power_user
5. "It's annoying that I have to restart it every time I want to check where I parked" → casual_user
6. "Decent app but needs improvement" → unclear

RESPONSE FORMAT:
Respond with ONLY ONE WORD: power_user, casual_user, or unclear (lowercase, no punctuation).
"""

    try:
        response = run_ollama(prompt, model_name).strip().lower()
        
        # Validate response
        valid_types = ["power_user", "casual_user", "unclear"]
        if response in valid_types:
            return response
        
        # Handle common variations
        if "power" in response:
            return "power_user"
        if "casual" in response:
            return "casual_user"
                
        # Default if response is invalid
        logging.warning(f"Invalid usage profile response: '{response}'. Defaulting to 'unclear'")
        return "unclear"
    except Exception as e:
        logging.error(f"Usage profile classification failed: {str(e)}")
        return "unclear"


def classify_pain_point(review_text: str, model_name: str) -> str:
    """
    Determine if the review mentions a pain point (yes/no).
    
    Args:
        review_text: The text of the review
        model_name: Name of the Ollama model to use
        
    Returns:
        str: "yes" or "no"
    """
    if not review_text or not isinstance(review_text, str):
        return "no"
        
    prompt = f"""You are an expert at analyzing BMW app reviews. Determine if this review mentions any pain points:

"{review_text}"

CLASSIFICATION TASK:
Determine if the user mentions any pain points, issues, or problems with the app.

GUIDELINES:
Classify as "yes" if the review mentions:
- Crashes, bugs, glitches, or technical issues
- Features not working as expected
- Frustration or difficulty using the app
- Complaints about design or performance
- Connection failures or syncing problems
- Missing expected functionality
- Errors or unexpected behavior
- Battery drain or other resource issues
- Anything the user clearly finds problematic

Classify as "no" if:
- The review is generally positive
- No specific issues or problems are mentioned
- The user is only making general comments or asking questions
- The review only contains feature requests without complaints

EXAMPLES:
1. "App keeps crashing when I try to check status" → yes
2. "Works great every time!" → no
3. "Why is it so hard to find the charging settings?" → yes
4. "Would be nice if you could add a widget" → no (feature request without complaint)
5. "Can't connect to my car half the time. Very frustrating!" → yes
6. "Just got the app. Looking forward to using it." → no

RESPONSE FORMAT:
Respond with ONLY ONE WORD: yes or no (lowercase, no punctuation).
"""

    try:
        response = run_ollama(prompt, model_name).strip().lower()
        
        # Validate response
        if response in ["yes", "no"]:
            return response
            
        # Handle potential extra text
        if "yes" in response:
            return "yes"
        if "no" in response:
            return "no"
                
        # Default if response is invalid
        logging.warning(f"Invalid pain point response: '{response}'. Defaulting to 'no'")
        return "no"
    except Exception as e:
        logging.error(f"Pain point classification failed: {str(e)}")
        return "no"


def classify_feature_request(review_text: str, model_name: str) -> str:
    """
    Determine if the review contains a feature request (yes/no).
    
    Args:
        review_text: The text of the review
        model_name: Name of the Ollama model to use
        
    Returns:
        str: "yes" or "no"
    """
    if not review_text or not isinstance(review_text, str):
        return "no"
        
    prompt = f"""You are an expert at analyzing BMW app reviews. Determine if this review contains a feature request:

"{review_text}"

CLASSIFICATION TASK:
Determine if the user is explicitly asking for or suggesting new features or improvements.

GUIDELINES:
Classify as "yes" if the review:
- Explicitly asks for a new feature to be added
- Suggests improvements to existing functionality
- Uses phrases like "would be nice if", "wish it had", "please add"
- Compares to missing features in other apps they want implemented
- Describes functionality they want but that doesn't exist yet
- Makes specific suggestions for changes or additions
- Expresses desire for missing capabilities

Classify as "no" if:
- The review doesn't suggest any improvements or new features
- The user is only reporting bugs or issues with existing features
- The user is only describing current functionality
- The review only contains complaints without suggesting solutions

EXAMPLES:
1. "Would be great if you could add Apple Watch support" → yes
2. "The app keeps crashing" → no
3. "Please add the ability to schedule charging" → yes
4. "Why can't I see my trip history like in the Mercedes app?" → yes
5. "The UI is terrible and confusing" → no (complaint without suggestion)
6. "You should include a way to share my location with family members" → yes
7. "I wish there was a widget for quick access" → yes
8. "Works great" → no

RESPONSE FORMAT:
Respond with ONLY ONE WORD: yes or no (lowercase, no punctuation).
"""

    try:
        response = run_ollama(prompt, model_name).strip().lower()
        
        # Validate response
        if response in ["yes", "no"]:
            return response
            
        # Handle potential extra text
        if "yes" in response:
            return "yes"
        if "no" in response:
            return "no"
                
        # Default if response is invalid
        logging.warning(f"Invalid feature request response: '{response}'. Defaulting to 'no'")
        return "no"
    except Exception as e:
        logging.error(f"Feature request classification failed: {str(e)}")
        return "no"


def extract_competitor(review_text: str, model_name: str) -> str:
    """
    Extract which competitor brands are mentioned in the review.
    """
    if not review_text or not isinstance(review_text, str):
        return "none"
        
    prompt = f"""You are an expert at analyzing BMW app reviews. Identify any competitor car brands mentioned in this review:

"{review_text}"

CLASSIFICATION TASK:
Determine if the user mentions any BMW competitors and extract the specific competitor brand name(s).

DETAILED GUIDELINES:
- Extract ONLY car manufacturer brands EXPLICITLY mentioned in the review
- DO NOT infer or guess competitors that aren't directly mentioned
- If NO competitor is mentioned, return "none"
- [additional guidelines remain the same]

CRITICAL INSTRUCTION:
Only return a competitor name if it EXPLICITLY appears in the review text. Do not hallucinate brands.
"""

    try:
        response = run_ollama(prompt, model_name).strip().lower()
        
        # Clean response - remove any punctuation except commas
        response = response.replace('"', '').replace("'", '').replace(".", "").replace("!", "").replace("?", "")
        
        if "none" in response or not response:
            return "none"
        
        # VERIFICATION: Check if the response actually appears in the original text
        review_lower = review_text.lower()
        
        # Check each competitor name in the response
        competitors = response.split(',')
        verified_competitors = []
        
        for competitor in competitors:
            # Common name variations
            variations = {
                "mercedes": ["mercedes", "merc", "mercedes-benz", "mercedes benz"],
                "volkswagen": ["volkswagen", "vw", "volkswagon"],
                "chevrolet": ["chevrolet", "chevy"]
            }
            
            # Check if this competitor or its variations appear in the text
            if competitor in review_lower:
                verified_competitors.append(competitor)
                continue
                
            # Check variations if available
            if competitor in variations:
                for variation in variations[competitor]:
                    if variation in review_lower:
                        verified_competitors.append(competitor)
                        break
        
        if verified_competitors:
            return ",".join(verified_competitors)
        else:
            return "none"
            
    except Exception as e:
        logging.error(f"Competitor extraction failed: {str(e)}")
        return "none"


def analyze_review_step_by_step(review_text: str, model_name: str) -> Dict:
    """
    Analyze a review by performing each classification task separately.
    
    Args:
        review_text: The text of the review
        model_name: Name of the Ollama model to use
        
    Returns:
        Dict: Dictionary with all classification results
    """
    # Ensure the review text is a string
    if not isinstance(review_text, str):
        review_text = str(review_text) if review_text is not None else ""
    
    # Process each classification in sequence
    sentiment = classify_sentiment(review_text, model_name)
    topics = classify_topics(review_text, model_name)
    vehicle_type = classify_vehicle_type(review_text, model_name)
    user_experience = classify_user_experience(review_text, model_name)
    usage_profile = classify_usage_profile(review_text, model_name)
    is_pain_point = classify_pain_point(review_text, model_name)
    is_feature_request = classify_feature_request(review_text, model_name)
    competitor_mentioned = extract_competitor(review_text, model_name)
    
    # Return all results in a dictionary
    return {
        "sentiment": sentiment,
        "topics": topics,
        "vehicle_type": vehicle_type,
        "user_experience": user_experience,
        "usage_profile": usage_profile,
        "is_pain_point": is_pain_point,
        "is_feature_request": is_feature_request,
        "competitor_mentioned": competitor_mentioned
    }


def process_reviews_step_by_step(df: pd.DataFrame, model_name: str, batch_size: int = 10, 
                                start_batch: int = 1) -> pd.DataFrame:
    """
    Process all reviews with step-by-step individual classifications.
    
    Args:
        df: DataFrame with reviews in 'content_english' column
        model_name: Name of the Ollama model to use
        batch_size: Number of reviews to process per batch
        start_batch: Which batch to start processing from (for resuming)
        
    Returns:
        DataFrame with all classification results added
    """
    if 'content_english' not in df.columns:
        raise ValueError("Input DataFrame must contain 'content_english' column")
    
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()
    total_reviews = len(result_df)
    total_batches = (total_reviews + batch_size - 1) // batch_size
    
    # Initialize columns with default values (only if starting from the beginning)
    if start_batch == 1:
        result_df['sentiment'] = 'neutral'
        result_df['topics'] = 'other'
        result_df['vehicle_type'] = 'unclear'
        result_df['user_experience'] = 'unclear' 
        result_df['usage_profile'] = 'unclear'
        result_df['is_pain_point'] = 'no'
        result_df['is_feature_request'] = 'no'
        result_df['competitor_mentioned'] = 'none'
    
    logging.info(f"Starting step-by-step analysis from batch {start_batch}/{total_batches}")
    
    # Keep track of checkpoint filenames
    checkpoint_files = []
    
    # Process in batches
    for batch_num in range(start_batch, total_batches + 1):
        start_idx = (batch_num - 1) * batch_size
        end_idx = min(start_idx + batch_size, total_reviews)
        batch_indices = result_df.index[start_idx:end_idx]
        
        logging.info(f"Processing batch {batch_num}/{total_batches} (reviews {start_idx+1}-{end_idx})")
        
        # Process each review in the batch
        for idx in tqdm(batch_indices, desc=f"Batch {batch_num}", unit="review"):
            review_text = result_df.loc[idx, 'content_english']
            
            # Skip empty reviews
            if pd.isna(review_text) or not str(review_text).strip():
                logging.warning(f"Skipping empty review at index {idx}")
                continue
            
            # Run all classifications
            try:
                results = analyze_review_step_by_step(str(review_text), model_name)
                
                # Update the DataFrame with results
                result_df.loc[idx, 'sentiment'] = results.get('sentiment', 'neutral')
                result_df.loc[idx, 'topics'] = results.get('topics', 'other')
                result_df.loc[idx, 'vehicle_type'] = results.get('vehicle_type', 'unclear')
                result_df.loc[idx, 'user_experience'] = results.get('user_experience', 'unclear')
                result_df.loc[idx, 'usage_profile'] = results.get('usage_profile', 'unclear')
                result_df.loc[idx, 'is_pain_point'] = results.get('is_pain_point', 'no')
                result_df.loc[idx, 'is_feature_request'] = results.get('is_feature_request', 'no')
                result_df.loc[idx, 'competitor_mentioned'] = results.get('competitor_mentioned', 'none')
                
            except Exception as e:
                logging.error(f"Error processing review at index {idx}: {e}")
                # Keep default values for this review
        
        # Save checkpoint after each batch
        checkpoint_filename = os.path.join(CHECKPOINT_DIR, f"batch_{batch_num}_of_{total_batches}.csv")
        batch_df = result_df.iloc[start_idx:end_idx].copy()
        batch_df.to_csv(checkpoint_filename, index=False)
        checkpoint_files.append(checkpoint_filename)
        logging.info(f"Saved checkpoint to {checkpoint_filename}")
        
        # Save progress information
        progress = {
            "last_completed_batch": batch_num,
            "total_batches": total_batches,
            "batch_size": batch_size,
            "total_reviews": total_reviews,
            "model_name": model_name,
            "last_processed_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        with open(PROGRESS_FILE, 'w') as f:
            json.dump(progress, f, indent=4)
        
        # Ask user if they want to continue
        if batch_num < total_batches:
            continue_processing = input(f"\nBatch {batch_num}/{total_batches} completed. Continue to next batch? (y/n): ")
            if continue_processing.lower() != 'y':
                logging.info(f"Processing paused after batch {batch_num}. Run again to continue from batch {batch_num + 1}.")
                return result_df  # Return the partially processed DataFrame
    
    # All batches completed, merge results
    logging.info("All batches completed. Merging results...")
    merge_checkpoints()
    
    return result_df

def merge_checkpoints():
    """Merge all checkpoint files into one consolidated file"""
    checkpoint_files = sorted(glob.glob(os.path.join(CHECKPOINT_DIR, "batch_*.csv")))
    
    if not checkpoint_files:
        logging.warning("No checkpoint files found to merge")
        return
    
    # Read and combine all checkpoint files
    dfs = []
    for file in checkpoint_files:
        try:
            df = pd.read_csv(file)
            dfs.append(df)
            logging.info(f"Added {file} to merge list ({len(df)} rows)")
        except Exception as e:
            logging.error(f"Error reading {file}: {e}")
    
    if not dfs:
        logging.error("No valid checkpoint files could be read")
        return
    
    # Concatenate all dataframes
    merged_df = pd.concat(dfs, ignore_index=True)
    
    # Save consolidated file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    consolidated_filename = os.path.join(RESULTS_DIR, f"bmw_reviews_consolidated_{timestamp}.csv")
    
    # Save full results
    merged_df.to_csv(consolidated_filename, index=False)
    logging.info(f"Saved consolidated results with {len(merged_df)} reviews to {consolidated_filename}")
    
    # Print quick summary
    print(f"\n=== Classification Summary ({len(merged_df)} reviews) ===")
    for col in ['sentiment', 'vehicle_type', 'user_experience', 'usage_profile', 
               'is_pain_point', 'is_feature_request', 'competitor_mentioned']:
        print(f"\n{col.replace('_', ' ').title()} distribution:")
        print(merged_df[col].value_counts())

def run_analysis(df, model_name, batch_size=50):
    """Main function to run or resume analysis"""
    start_batch = 1
    total_batches = (len(df) + batch_size - 1) // batch_size
    
    # Check if we have a progress file to resume from
    if os.path.exists(PROGRESS_FILE):
        try:
            with open(PROGRESS_FILE, 'r') as f:
                progress = json.load(f)
            
            last_batch = progress.get("last_completed_batch", 0)
            saved_total_batches = progress.get("total_batches", 0)
            prev_model = progress.get("model_name", "")
            last_time = progress.get("last_processed_time", "unknown time")
            
            if last_batch < total_batches:
                print(f"Previous run found (completed {last_batch}/{saved_total_batches} batches at {last_time}).")
                
                # Let user choose which batch to start from
                print(f"\nBatch information:")
                print(f"- Total batches: {total_batches}")
                print(f"- Completed batches: 1 to {last_batch}")
                print(f"- Remaining batches: {last_batch + 1} to {total_batches}")
                
                while True:
                    batch_input = input(f"\nEnter the batch number to start from (1-{total_batches}) or 'q' to quit: ")
                    
                    if batch_input.lower() == 'q':
                        logging.info("Analysis cancelled by user")
                        return None
                    
                    try:
                        selected_batch = int(batch_input)
                        if 1 <= selected_batch <= total_batches:
                            start_batch = selected_batch
                            logging.info(f"Starting from user-selected batch {start_batch}")
                            
                            # Warn if starting from an incomplete batch
                            if selected_batch <= last_batch:
                                overwrite = input(f"Batch {selected_batch} was already completed. Reprocess this batch? (y/n): ")
                                if overwrite.lower() != 'y':
                                    # User changed their mind - ask again
                                    continue
                            
                            # Warn if model changed
                            if prev_model and prev_model != model_name:
                                logging.warning(f"Using a different model ({model_name}) than previous run ({prev_model})")
                            
                            break
                        else:
                            print(f"Invalid batch number. Please enter a number between 1 and {total_batches}.")
                    except ValueError:
                        print("Please enter a valid number.")
            else:
                print("Previous run completed all batches. Starting over.")
        except Exception as e:
            logging.error(f"Error reading progress file: {e}")
            print(f"Error reading progress file: {e}")
    else:
        print("No previous run found. Starting from the beginning.")
    
    # Start or resume processing
    print(f"Starting classification from batch {start_batch}...")
    start_time = time.time()
    
    df_classified = process_reviews_step_by_step(
        df=df,
        model_name=model_name,
        batch_size=batch_size,
        start_batch=start_batch
    )
    
    # Calculate elapsed time
    total_time = time.time() - start_time
    hours, remainder = divmod(total_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Classification complete! Time taken: {int(hours)}h {int(minutes)}m {int(seconds)}s")
    
    return df_classified

In [11]:
# Run the full analysis (with resume capability)
df_classified = run_analysis(
    df=df2,
    model_name=ollama_model_name,
    batch_size=5 # Process in batches of 5
)

# Final output is already saved as part of run_analysis
print("Classification complete!")

Previous run found (completed 2/5 batches at 2025-04-09 12:48:26).

Batch information:
- Total batches: 5
- Completed batches: 1 to 2
- Remaining batches: 3 to 5


2025-04-09 14:45:22,785 - INFO - Starting from user-selected batch 1
2025-04-09 14:45:27,427 - INFO - Starting step-by-step analysis from batch 1/5
2025-04-09 14:45:27,428 - INFO - Processing batch 1/5 (reviews 1-5)


Starting classification from batch 1...


Batch 1: 100%|██████████| 5/5 [01:06<00:00, 13.27s/review]
2025-04-09 14:46:33,772 - INFO - Saved checkpoint to bmw_app_analysis/checkpoints/batch_1_of_5.csv
2025-04-09 14:46:50,467 - INFO - Processing batch 2/5 (reviews 6-10)
Batch 2: 100%|██████████| 5/5 [01:07<00:00, 13.52s/review]
2025-04-09 14:47:58,058 - INFO - Saved checkpoint to bmw_app_analysis/checkpoints/batch_2_of_5.csv
2025-04-09 14:48:02,696 - INFO - Processing batch 3/5 (reviews 11-15)
Batch 3: 100%|██████████| 5/5 [01:10<00:00, 14.02s/review]
2025-04-09 14:49:12,867 - INFO - Saved checkpoint to bmw_app_analysis/checkpoints/batch_3_of_5.csv
2025-04-09 14:50:31,599 - INFO - Processing batch 4/5 (reviews 16-20)
Batch 4: 100%|██████████| 5/5 [01:09<00:00, 13.81s/review]
2025-04-09 14:51:40,679 - INFO - Saved checkpoint to bmw_app_analysis/checkpoints/batch_4_of_5.csv
2025-04-09 14:51:48,620 - INFO - Processing batch 5/5 (reviews 21-25)
Batch 5: 100%|██████████| 5/5 [01:11<00:00, 14.33s/review]
2025-04-09 14:53:00,295 - INFO


=== Classification Summary (25 reviews) ===

Sentiment distribution:
sentiment
positive    9
negative    9
neutral     7
Name: count, dtype: int64

Vehicle Type distribution:
vehicle_type
unclear       22
ev_hybrid      2
combustion     1
Name: count, dtype: int64

User Experience distribution:
user_experience
unclear             22
experienced_user     3
Name: count, dtype: int64

Usage Profile distribution:
usage_profile
unclear        23
power_user      1
casual_user     1
Name: count, dtype: int64

Is Pain Point distribution:
is_pain_point
yes    13
no     12
Name: count, dtype: int64

Is Feature Request distribution:
is_feature_request
no     20
yes     5
Name: count, dtype: int64

Competitor Mentioned distribution:
competitor_mentioned
none    25
Name: count, dtype: int64
Classification complete! Time taken: 0h 7m 32s
Classification complete!


In [13]:
def analyze_classification_results(df_classified, display_specific_review=None):
    """
    Analyze and display classification results with summary statistics 
    and optionally show specific reviews.
    
    Args:
        df_classified: DataFrame with classified reviews
        display_specific_review: Index of specific review to display (optional)
    """
    # Import display from IPython if available, otherwise use print as fallback
    try:
        from IPython.display import display, HTML
    except ImportError:
        # Define a simple display function if not in a notebook environment
        def display(obj):
            print(obj)
    
    if df_classified is None or len(df_classified) == 0:
        print("No classification results to analyze.")
        return
    
    # Define the display columns for consistency
    display_columns = [
        'content_english', 'sentiment', 'topics', 
        'is_pain_point', 'is_feature_request', 'competitor_mentioned',
        'vehicle_type', 'user_experience', 'usage_profile'
    ]
    
    # If a specific review is requested, display it first
    if display_specific_review is not None:
        try:
            review_idx = int(display_specific_review)
            if 0 <= review_idx < len(df_classified):
                print(f"\n=== Review #{review_idx+1} (Index {review_idx}) ===")
                selected_review = df_classified.iloc[review_idx]
                display(selected_review[display_columns])
                
                print(f"\n=== Review #{review_idx+1} (Detailed) ===")
                for col in display_columns:
                    print(f"{col}: {selected_review[col]}")
            else:
                print(f"Review index {review_idx} is out of range (0-{len(df_classified)-1}).")
        except ValueError:
            print("Please provide a valid review index (number).")
    
    # Display sample of classified reviews (first 5 by default)
    print("\n=== Sample Classified Reviews ===")
    display(df_classified[display_columns].head())
    
    # Display classification summary statistics
    print(f"\n=== Classification Summary ({len(df_classified)} reviews) ===")
    
    # Helper function to show value distributions with percentages
    def show_distribution(df, column):
        counts = df[column].value_counts()
        percentages = df[column].value_counts(normalize=True) * 100
        distribution = pd.DataFrame({
            'Count': counts,
            'Percentage': percentages.round(1)
        })
        print(f"\n--- {column.replace('_', ' ').title()} Distribution ---")
        display(distribution)
    
    # Show distributions for each classification
    show_distribution(df_classified, 'sentiment')
    show_distribution(df_classified, 'vehicle_type')
    show_distribution(df_classified, 'user_experience')
    show_distribution(df_classified, 'usage_profile')
    show_distribution(df_classified, 'is_pain_point')
    show_distribution(df_classified, 'is_feature_request')
    show_distribution(df_classified, 'competitor_mentioned')
    
    # For topics, we need to split and count individually
    print("\n--- Topics Distribution ---")
    all_topics = [topic.strip() for topics_str in df_classified['topics'].dropna() 
                  for topic in topics_str.split(',') if topic.strip()]
    topic_counts = pd.Series(all_topics).value_counts()
    display(topic_counts)
    
    print("\n=== Review Lookup ===")
    print("To view a specific review, run: analyze_classification_results(df_classified, review_index)")
    print("Where review_index is the index of the review you want to see (0 to", len(df_classified)-1, ")")

# When doing the full analysis:
if df_classified is not None:
    # Save results to CSV
    output_file = os.path.join(RESULTS_DIR, "bmw_reviews_classified.csv")
    df_classified.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    
    # Show initial summary statistics
    analyze_classification_results(df_classified)

Results saved to bmw_app_analysis/results/bmw_reviews_classified.csv

=== Sample Classified Reviews ===


,content_english,sentiment,topics,is_pain_point,is_feature_request,competitor_mentioned,vehicle_type,user_experience,usage_profile
7495,Top functions and a real added value.,positive,"ui/ux, vehicle status",no,no,none,unclear,unclear,unclear
18249,"The screen could also show current speed, gear...",neutral,"feature requests, vehicle status",no,yes,none,unclear,unclear,power_user
9574,Good,positive,other,no,no,none,unclear,unclear,unclear
251,Car location not updated any more - the car lo...,negative,"vehicle status, connectivity",yes,no,none,unclear,unclear,unclear
15356,"Very practical app, thanks to which you can pl...",positive,"trip planning, remote controls",no,no,none,unclear,unclear,unclear



=== Classification Summary (25 reviews) ===

--- Sentiment Distribution ---


,Count,Percentage
sentiment,,
positive,9,36.0
negative,9,36.0
neutral,7,28.0



--- Vehicle Type Distribution ---


...


--- Competitor Mentioned Distribution ---


,Count,Percentage
competitor_mentioned,,
none,25,100.0



--- Topics Distribution ---


vehicle status            9
performance               8
ui/ux                     6
remote controls           5
authentication            5
connectivity              4
updates                   3
feature requests          2
other                     2
charging management       2
service & maintenance     2
trip planning             1
digital key/mobile key    1
customer support          1
Name: count, dtype: int64


=== Review Lookup ===
To view a specific review, run: analyze_classification_results(df_classified, review_index)
Where review_index is the index of the review you want to see (0 to 24 )
